In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
from tensorflow.keras.callbacks import TensorBoard

In [7]:
from sentence_transformers import SentenceTransformer

# Replace 'sentence-transformers/bert-base-nli-mean-tokens' with your desired model name
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
model = SentenceTransformer(model_name)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df = pd.read_parquet('clean_data.pqt')

In [4]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww matches background colour seemingly stuck ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,make real suggestions improvement wondered sec...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,second time asking view completely contradicts...,0,0,0,0,0,0
159567,ffea4adeee384e90,ashamed horrible thing put talk page 128 61 19 93,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm theres actual article prostitution...,0,0,0,0,0,0
159569,fff125370e4aaaf3,looks like actually put speedy first version d...,0,0,0,0,0,0


In [5]:
mini = df.sample(400)

In [6]:
mini

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
142912,fc4ee980bb7db830,feel free add prime minister salvador allende ...,0,0,0,0,0,0
100219,188bc06318b5c088,waseem akhtar spectacularly bad speedy deletio...,0,0,0,0,0,0
123882,96a19853bfa3776e,end world bye bye say fun wanna block every li...,0,0,0,0,0,0
66389,b19c406eab9968d2,niggus luv dem wyt biches fa reel,1,0,0,0,0,0
80070,d63f44c59d8ee4bb,checked edit history stick editing wikipedia a...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
25829,445c9499fa70f034,made start seen think mistake made prior faile...,0,0,0,0,0,0
103146,27f86498382333de,keep updated made page dioptinae subfamily mad...,0,0,0,0,0,0
62655,a7a89728dfc59c40,oh thanks pointing extra section lead minutes ...,0,0,0,0,0,0
114248,63294f1e6215eba8,thanks yeah saw interested pressing point edit...,0,0,0,0,0,0


In [9]:
embeddings= model.encode(mini['comment_text'].tolist())
mini['emb'] = embeddings.tolist()

In [13]:
max_words= 1250
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.sequence import pad_sequences

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ResourceExhaustedError: Exception encountered when calling layer "self" "                 f"(type TFBertSelfAttention).

{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[400,12,459,459] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:BatchMatMulV2]

Call arguments received by layer "self" "                 f"(type TFBertSelfAttention):
  • hidden_states=tf.Tensor(shape=(400, 459, 768), dtype=float32)
  • attention_mask=tf.Tensor(shape=(400, 1, 1, 459), dtype=float32)
  • head_mask=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_value=None
  • output_attentions=False
  • training=False

In [ ]:
# filename= "ST_LSTM"
# tb = TensorBoard(log_dir= "logs\\{}".format(filename))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

i = Input(shape=(max_words, sentence_embeddings.shape[2]))
x = LSTM(128)(i)
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='softmax')(x)
lstm_model = Model(i, x) 


In [ ]:
lstm_model.compile(loss='categorical_crossentropy', optimizer= Adam(learning_rate=.000025), metrics=['accuracy', Recall(name= 'recall'), Precision(name='precision')])


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense

i = Input(shape=(sentence_embeddings.shape[1], sentence_embeddings.shape[2]))
x = Conv1D(128, kernel_size=3, activation='relu')(i)
x = GlobalMaxPooling1D()(x)
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='sigmoid')(x)
cnn_model = Model(i,x)
